### Fera formidável 4.2 - Stop right now, thank you very much 

#### Enunciado:
Objetivo: implemente uma estratégia de Parada Antecipada (Early Stopping) no processo de treino da rede neural feita em Python puro ou no processo de treino da rede neural feita em PyTorch

Comentário: esta não é para resolver com o módulo lightning

### Introdução

Essa fera 4.2 consiste em implementar a estratégia de Early Stopping em uma rede neural. A estratégia Early Stopping consiste em parar o treinamento da rede neural assim que o seu desempenho para dados de validação começa a diminuir, evitando o overfitting.

A imagem abaixo demonstra como funciona essa estratégia.

<img alt="Alt text" title="Early Stopping" src="Redes%20Neurais/Imagens/Early_stopping.png">


Passo a passo:

Treinar a rede por épocas.

Após cada época, avaliar no conjunto de validação.

Salvar o modelo se o desempenho melhorar.

Se não melhorar por um número patience de épocas consecutivas, parar o treinamento.

### Importações

In [189]:
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time

### Definindo classes

In [190]:
class MLP(nn.Module):
    def __init__(self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets):
        super().__init__()
        
        self.camadas = nn.Sequential(
            nn.Linear(num_dados_entrada, neuronios_c1),
            nn.Sigmoid(),
            nn.Linear(neuronios_c1, neuronios_c2),
            nn.Sigmoid(),
            nn.Linear(neuronios_c2, num_targets),
        )
        
    def forward(self, x):
        x = self.camadas(x)
        return x

### Definindo o dataset

In [191]:
df = sns.load_dataset("iris")
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


### Dividindo os dados em treino e teste

In [192]:
TAMANHO_TESTE = 0.1

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

In [193]:
df_treino

,sepal_length,sepal_width,petal_length,petal_width,species
24,4.8,3.4,1.9,0.2,setosa
122,7.7,2.8,6.7,2.0,virginica
89,5.5,2.5,4.0,1.3,versicolor
20,5.4,3.4,1.7,0.2,setosa
33,5.5,4.2,1.4,0.2,setosa
...,...,...,...,...,...
75,6.6,3.0,4.4,1.4,versicolor
47,4.6,3.2,1.4,0.2,setosa
98,5.1,2.5,3.0,1.1,versicolor
74,6.4,2.9,4.3,1.3,versicolor


In [194]:
# P/ treino da rede
x_treino = df_treino["petal_length"].values
y_verdadeiro_treino = df_treino["petal_width"].values

x_treino = torch.tensor(x_treino)
x_treino = x_treino.view(-1, 1)
y_verdadeiro_treino = torch.tensor(y_verdadeiro_treino)
y_verdadeiro_treino = y_verdadeiro_treino.view(-1, 1)

# P/ validação da rede
x_validacao = df_teste["petal_length"].values
y_verdadeiro_validacao = df_teste["petal_width"].values

x_validacao = torch.tensor(x_validacao)
x_validacao = x_validacao.view(-1, 1)
y_verdadeiro_validacao = torch.tensor(y_verdadeiro_validacao)
y_verdadeiro_validacao = y_verdadeiro_validacao.view(-1,1)

In [195]:
x_treino = x_treino.float()
y_verdadeiro_treino = y_verdadeiro_treino.float()

x_validacao = x_validacao.float()
y_verdadeiro_validacao = y_verdadeiro_validacao.float()

### Criando a MLP

In [196]:
NUM_DADOS_DE_ENTRADA = 1
NUM_DADOS_DE_SAIDA = 1
NEURONIOS_C1 = 3
NEURONIOS_C2 = 2

In [197]:
minha_mlp = MLP(
    NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA
)

In [198]:
y_prev = minha_mlp(x_treino)

TAXA_DE_APRENDIZADO = 0.005

otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)
fn_perda = nn.MSELoss()

### Implementando o Early Stopping

In [199]:
NUMERO_EPOCAS_SEM_MELHORA = 12
MAXIMO_EPOCAS = 100000

melhor_rmse = float('inf')
cont = 0
cont_maximo = 0
epoca_melhor_modelo = 0

inicio = time.time()

while cont < NUMERO_EPOCAS_SEM_MELHORA and cont_maximo < MAXIMO_EPOCAS:
    
    # Treino
    minha_mlp.train()

    y_pred = minha_mlp(x_treino)
    otimizador.zero_grad()
    loss = fn_perda(y_verdadeiro_treino, y_pred)
    loss.backward()
    otimizador.step()

    # Teste
    minha_mlp.eval()

    with torch.no_grad():
        y_pred = minha_mlp(x_validacao)
        
    RMSE = mean_squared_error(y_verdadeiro_validacao, y_pred, squared=False)
    
    if RMSE < melhor_rmse:
        melhor_rmse = RMSE
        cont = 0
        epoca_melhor_modelo = cont_maximo
        melhor_desempenho = torch.save(minha_mlp.state_dict(), 'checkpoint.pt')
        
    else:
        cont += 1

    cont_maximo += 1
    print(f"Época {cont_maximo} | RMSE: {RMSE:.4f}")
    
fim = time.time()
tempo = fim - inicio

print()
print(f"Tempo rodar o código: {tempo:.2f} segundos")

minha_mlp.load_state_dict(torch.load('checkpoint.pt'))

Época 1 | RMSE: 2.5068
Época 2 | RMSE: 2.4680
Época 3 | RMSE: 2.4301
Época 4 | RMSE: 2.3931
Época 5 | RMSE: 2.3570
Época 6 | RMSE: 2.3218
Época 7 | RMSE: 2.2875
Época 8 | RMSE: 2.2539
Época 9 | RMSE: 2.2212
Época 10 | RMSE: 2.1892
Época 11 | RMSE: 2.1580
Época 12 | RMSE: 2.1274
Época 13 | RMSE: 2.0976
Época 14 | RMSE: 2.0685


Época 15 | RMSE: 2.0400
Época 16 | RMSE: 2.0121
Época 17 | RMSE: 1.9849
Época 18 | RMSE: 1.9583
Época 19 | RMSE: 1.9323
Época 20 | RMSE: 1.9068
Época 21 | RMSE: 1.8819
Época 22 | RMSE: 1.8576
Época 23 | RMSE: 1.8338
Época 24 | RMSE: 1.8104
Época 25 | RMSE: 1.7876
Época 26 | RMSE: 1.7653
Época 27 | RMSE: 1.7435
Época 28 | RMSE: 1.7221
Época 29 | RMSE: 1.7012
Época 30 | RMSE: 1.6808
Época 31 | RMSE: 1.6607
Época 32 | RMSE: 1.6411
Época 33 | RMSE: 1.6219
Época 34 | RMSE: 1.6031
Época 35 | RMSE: 1.5847
Época 36 | RMSE: 1.5667
Época 37 | RMSE: 1.5491
Época 38 | RMSE: 1.5318
Época 39 | RMSE: 1.5149
Época 40 | RMSE: 1.4984
Época 41 | RMSE: 1.4822
Época 42 | RMSE: 1.4663
Época 43 | RMSE: 1.4508
Época 44 | RMSE: 1.4356
Época 45 | RMSE: 1.4207
Época 46 | RMSE: 1.4061
Época 47 | RMSE: 1.3918
Época 48 | RMSE: 1.3778
Época 49 | RMSE: 1.3641
Época 50 | RMSE: 1.3507
Época 51 | RMSE: 1.3376
Época 52 | RMSE: 1.3247
Época 53 | RMSE: 1.3122
Época 54 | RMSE: 1.2998
Época 55 | RMSE: 1.2878
Época 56 | RMSE:

<All keys matched successfully>

### Resultado obtido com o Early Stopping

In [200]:
print(f"O melhor modelo foi obtido na época {epoca_melhor_modelo}, com RMSE de {melhor_rmse:.4f}")

O melhor modelo foi obtido na época 21974, com RMSE de 0.1833


### Conclusão

